## N-Gram language models + Markov Models

Exercises exploring the use of the `nltk` library to generate different n-gram utilizations, and exploring applications of Markov Models.
_________________________

#### First, load the dataset:

As the first step towards imitating Shakespeare's writing, you will create a function called `load_data` that loads his original *Sonnets* from `assets/gutenberg/THE_SONNETS.txt`. This function should accomplish the following: 

* **Extract sentences from the data file.** Of course, depending on the nature of the task at hand, what constitutes a *sentence* can vary. In the context of this assignment, we will define a sentence as just a line of a sonnet, regardless of the punctuation at end. In addition, we will ignore the boundaries of the sonnets --- that is, we are not dealing with $154$ individual *sonnets* but rather $154 \times 14 = 2156$ *sentences* (actually only $2155$ sentences, as *Sonnet 99* has [15 lines](https://www.google.com/search?ei=po4hX9jzN4rKtQaL7K-wDg&q=why+is+sonnet+99+15+lines&oq=why+is+sonnet+99+15+lines&gs_lcp=CgZwc3ktYWIQAzoECAAQR1ChGlihGmCqHGgAcAJ4AIABXIgBXJIBATGYAQCgAQGqAQdnd3Mtd2l6wAEB&sclient=psy-ab&ved=0ahUKEwjY3snX3PLqAhUKZc0KHQv2C-YQ4dUDCAw&uact=5) but *Sonnet 126* has only [12](https://www.google.com/search?ei=d4whX6_uH9a4tQbUiISoDA&q=why+is+sonnet+126+only+12+lines&oq=o+thou+my+lovely+boy+who+in+thy+power&gs_lcp=CgZwc3ktYWIQARgBMgQIABBHMgQIABBHMgQIABBHMgQIABBHMgQIABBHMgQIABBHMgQIABBHMgQIABBHUABYAGCWEWgAcAJ4AIABAIgBAJIBAJgBAKoBB2d3cy13aXrAAQE&sclient=psy-ab)). Finally, make sure that the newline character `\n` at the end of each line is dropped. 


* **Tokenise each extracted sentence.** While it's ambiguous what a sentence is, what constitutes a "*word*" is even more task-dependent. Do punctuations count as "words"? Are two "words" with the same spelling but different casing considered identical? Since what a text file contains is nothing more than a squence of characters, there are many possible ways of grouping these characters to form "words" that are subsequently taken as input by a program. To distinguish what's actually taken as input by a program from a linguistic word, we call the former a *token*. The process of producing a list of tokens out of a sentence is then called *tokenisation*. At this step, you will first lower-case each sentence extracted from the previous step entirely and then tokenise each lower-cased sentence. You may use any tokeniser of your choice, such as `word_tokenize` from the `nltk` library. 

In [1]:
# configure nltk
import nltk

nltk_data_path = "assets/nltk_data"
if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)

In [2]:
def load_data():
    """
    Load text data from a file and produce a list of token lists
    """
    
    sentences = []
    
    filename = '../assets/gutenberg/THE_SONNETS.txt'
    
    f = open(filename,'r')
    
    for line in f.readlines():
        tokens = nltk.tokenize.word_tokenize(line.lower())
        if len(tokens) > 1:
            sentences.append(tokens)
    
    return sentences

#len(load_data()) #2155
print(load_data()[0])
print(load_data()[1])
print('...')
print(load_data()[-2])
print(load_data()[-1])

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', ',']
['that', 'thereby', 'beauty', '’', 's', 'rose', 'might', 'never', 'die', ',']
...
['came', 'there', 'for', 'cure', 'and', 'this', 'by', 'that', 'i', 'prove', ',']
['love', '’', 's', 'fire', 'heats', 'water', ',', 'water', 'cools', 'not', 'love', '.']


#### Build vocabulary of unique tokens

Will use `<s>` and `</s>` in vocabulary because we are using them to pad n-gram language models. Token storage order will not matter.

In [3]:
def build_vocab(sentences):
    """
    Take a list of sentences and return a vocab
    """
    
    vocab = []
    
    result = {x for l in sentences for x in l}
    result = list(result)
    result.extend(['<s>', '</s>'])
    
    return result

#### Generate all n-grams

Two-step process:

* **Pad each sentence with `<s>` and `</s>` for $n \geq 2$**.
* **Generate $n$-grams on the padded sentences**.

In [4]:
def build_ngrams(n, sentences):
    """
    Take a list of unpadded sentences and create all n-grams as specified by the argument "n" for each sentence
    """
    
    all_ngrams = []
    
    for sent in sentences:
        if len(sent) >= 2:
            new_sent = list(nltk.lm.preprocessing.pad_both_ends(sent,n))
            all_ngrams.append(list(nltk.ngrams(new_sent,n)))
    
    return all_ngrams

In [5]:
res_n = 4
res_sents = load_data()
res_ngrams = build_ngrams(res_n, res_sents)
res_ngrams[0]

[('<s>', '<s>', '<s>', 'from'),
 ('<s>', '<s>', 'from', 'fairest'),
 ('<s>', 'from', 'fairest', 'creatures'),
 ('from', 'fairest', 'creatures', 'we'),
 ('fairest', 'creatures', 'we', 'desire'),
 ('creatures', 'we', 'desire', 'increase'),
 ('we', 'desire', 'increase', ','),
 ('desire', 'increase', ',', '</s>'),
 ('increase', ',', '</s>', '</s>'),
 (',', '</s>', '</s>', '</s>')]

_____________
## Token Prediction with Markov Chains
Let's train an n-gram language model that will "imitate" William Shakespeare's writing.

First, let's assume we are working with bi-grams here (first-order Markov).

In [6]:
# run functions
sentence_data = load_data()
vocab = build_vocab(sentence_data)
bigrams = build_ngrams(2, sentence_data)

In [7]:
# this is my first iteration of this. It is more step-by-step

def bigram_next_token(start_tokens=("<s>", ) * 3):
    """
    Take some starting tokens and produce the most likely token that follows under a bi-gram model
    """
    
    from collections import Counter
    
    n = 2
    sents = load_data()
    bigrams = build_ngrams(n, sents)
    
    prev_token = start_tokens[-1]
    
    cnt_lst = [x for b in bigrams for x in set(b) if x[0] == prev_token]
    c1 = Counter(cnt_lst)
    
    tot = 0
    for k in c1:
        tot += c1[k]
    
    prob = 0
    next_token = ''
    for ele in c1:
        k = ele[-1]
        v = c1[ele] / tot
        #dct[k] = v
        if v > prob:
            prob = v
            next_token = k
    
    return next_token, prob

In [8]:
# Function test for after <s>,<s>,<s>,...
bigram_next_token(start_tokens=("<s>", ) * 3)

('and', 0.1122969837587007)

In [9]:
# This is a later, cleaner iteration

def bigram_next_token(start_tokens=("<s>", ) * 3):
    """
    Take some starting tokens and produce the most likely token that follows under a bi-gram model
    """
    global bigrams
    
    bigram_occurences = [grams[0] for grams in bigrams]
    bigram_set = set(bigram_occurences)
    start_count = len(bigram_occurences)
    bigram_dict = {}
    
    
    for bi in bigram_set:
        bigram_dict[bi] = bigram_occurences.count(bi)/start_count

    keymax = max(bigram_dict, key=bigram_dict.get)

    next_token, prob = keymax[1], bigram_dict[keymax]
    
    return next_token, prob

In [10]:
# Should result in the same output
bigram_next_token(start_tokens=("<s>", ) * 3)

('and', 0.1122969837587007)

#### Train an $n$-gram language model

Leverage `MLE` class from `nltk.lm`. Will require list of all n-grams for each sentence (`build_ngrams(n, sents)` and a vocabulary (`build_vocab(sents)`).


In [11]:
from nltk.lm import MLE

def train_ngram_lm(n):
    """
    Train a n-gram language model as specified by the argument "n"
    """
    
    lm = MLE(n)
    
    sents = load_data()
    train = build_ngrams(n, sents)
    vocabulary = build_vocab(sents)
    
    # https://www.kite.com/python/docs/nltk.lm
    lm.fit(train, vocabulary)
    
    return lm

#### Set sonnet parameters and write a sonnet using code:

In [12]:
# Every time it runs, depending on how drunk it is, a different sonnet is written. 

# Sonnet criteria:
n = 3
num_lines = 14
num_words_per_line = 8
text_seed = ["<s>"] * (n - 1)

lm = train_ngram_lm(n)

# Sonnet model
sonnet = []
while len(sonnet) < num_lines:
    while True:  # keep generating a line until success
        try:
            line = lm.generate(num_words_per_line, text_seed=text_seed)
        except ValueError:  # the generation is not always successful. need to capture exceptions
            continue
        else:
            line = [x for x in line if x not in ["<s>", "</s>"]]
            sonnet.append(" ".join(line))
            break

# print sonnet
print("\n".join(sonnet))

which vulgar scandal stamped upon my self almost
with means more blessed than my barren rhyme
of him i lose thee ,
when wasteful war shall statues overturn ,
nor taste , nor despised ,
to one , hath every one , one
if i no more be grieved at that
and with old woes new wail my dear
at my mistress ’ eyes .
lest sorrow lend me words and words express
of others ’ works thou dost wake elsewhere
and maiden virtue rudely strumpeted ,
herein lives wisdom , beauty doth he give
or bends with the bett ’ ring things


#### Hidden Markov Models for Part-of-Speech (POS) Tagging

First, load POS-tagged data for training and testing. The data comes from a [CoNLL-2003 shared task](https://www.clips.uantwerpen.be/conll2003/ner/). The shared task was originally held as a competition for Named Entity Recognition (NER), but the data it provided also includes POS tags that make POS Tagging possible. NER and POS Tagging are very similar tasks and HMMs are capable of handling both of them.

In [13]:
def load_sent_data(data_file, label=True):
    """
    Load tokens (and labels, if allowed) from a data_file
    """
    
    tagged_sents = []
    
    f = open(data_file, 'r')
    f = f.read().split('\n\n')
  
    if label == True:
        tagged_sents = [[(s.split()[0], s.split()[1]) for s in lst if len(s.split()) > 0] for lst in [sent.split('\n') for sent in f]]
        tagged_sents = [lst for lst in tagged_sents if len(lst) >= 10]
        
    elif label == False:
        tagged_sents = [[s.split()[0] for s in lst if len(s.split()) > 0] for lst in [sent.split('\n') for sent in f]]
        tagged_sents = [lst for lst in tagged_sents if len(lst) >= 10]
    
    return tagged_sents


# load_data('../assets/conll03/eng.train', label=True)
# load_data('../assets/conll03/eng.testa', label=True)
# load_data('../assets/conll03/eng.testb', label=False)

#### Create a training, development and testing sets using load_sent_data above
In all the three data files, we only consider "substantial" sentences that have at least 10 tokens. Labels <i>not</i> included in the testing set.

In [14]:
dataset = {"train": load_sent_data('../assets/conll03/eng.train', label=True), 
           "dev": load_sent_data('../assets/conll03/eng.testa', label=True), 
           "test": load_sent_data('../assets/conll03/eng.testb', label=False)
          }

#### Train HMM tagger
This cell may take a minute. The `HiddenMarkovModelTagger` function returns the accuracy on the development set.

In [15]:
# %%time

from nltk.tag.hmm import HiddenMarkovModelTagger

hmm_tagger = HiddenMarkovModelTagger.train(labeled_sequence=dataset['train'], 
                                           test_sequence=dataset['dev']
                                           )

accuracy over 43319 tokens: 89.13


The accuracy is 89.13%. Not bad. Not perfect.

Apply it to testing set to see how it does. THe line below tags the first sentence in the testing set.

In [16]:
hmm_tagger.tag(dataset["test"][0])

[('SOCCER', 'NN'),
 ('-', ':'),
 ('JAPAN', 'NNP'),
 ('GET', '.'),
 ('LUCKY', '"'),
 ('WIN', 'NNP'),
 (',', ','),
 ('CHINA', 'NNP'),
 ('IN', 'IN'),
 ('SURPRISE', 'NNP'),
 ('DEFEAT', 'NNP'),
 ('.', '.')]

______________________
<div style="text-align: right"><sub>Exercise adapted and modified from UMSI homework assignment for SIADS 632.</sub></div>